### geocode


In [20]:
import numpy as np

radarWavelength=0.238403545

In [ ]:
!geocode.py -h

In [ ]:
#method 1
!geocode.py ./Igrams/*/filt*.unw -l ./inputs/geometryRadar.h5 
!geocode.py ./Igrams/*/filt*.cor -l ./inputs/geometryRadar.h5 



In [ ]:
#method 2 (either this or method 1)
# !geocode.py ./Igrams/*/filt*.unw --lat-file ./geom_reference/geometry8x3/lat.rdr --lon-file ./geom_reference/geometry8x3/lon.rdr
# !geocode.py ./Igrams/*/filt*.cor --lat-file ./geom_reference/geometry8x3/lat.rdr --lon-file ./geom_reference/geometry8x3/lon.rdr



In [ ]:
#generate cor mask
!generate_mask.py geo*cor -m 0.7 -o geo_mask_CorPt7.h5

In [ ]:
!reference_point.py  geo_filt_20230725_20230925_snaphu.unw  -l  39.242805  -L -106.497245  -m geo_mask_CorPt7.h5

In [ ]:
!save_kmz.py geo_filt_20230725_20230925_snaphu.unw -m geo_mask_CorPt7.h5 -v -5 5

In [ ]:
!mv geo*filt*unw* ./Igrams/20230725_20230925/
!mv geo*filt*cor* ./Igrams/20230725_20230925/
!mv geo*mask*.h5* ./Igrams/20230725_20230925/

In [ ]:
# gdal_translate -b 1 -of in_image output_image.tif
# gdal_translate -of Gtiff in_image output_image.tif


In [ ]:
# Read the unwrapped file
src = rioxarray.open_rasterio(f'./unwrapped/{ref}_{sec}.unw.tif')
merged_unw = src.rio.reproject("EPSG:4326")[0] * wavelength / (-4 * np.pi)
src = []

# Plot using Matplotlib
fig, ax = plt.subplots(figsize=[12,9])
merged_unw_cm = merged_unw*100
merged_unw_cm -= merged_unw_cm[refy, refx]              # Reference the data to a point
cax = ax.imshow(merged_unw_cm, cmap='RdBu_r',interpolation='nearest', origin='upper', extent=bbox, vmin=np.min(merged_unw_cm), vmax=-np.min(merged_unw_cm))
ax.plot(reflon, reflat, 'ks')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title(f'UNW_{ref}-{sec}_{orbitPass[0]}{pathNumber}',fontsize=12)
cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
cbar.set_label('LOS Displacement (cm)', rotation=270, labelpad=15)
plt.show()